# **Transformer Architecture**

### **Encoder-Decoder Architecture**

It is used for **sequence-to-sequence** tasks like machine translation.

This is the architecture proposed by the original transformer paper [Attention is All You Need](https://arxiv.org/abs/1706.03762) by Vaswani et al. in 2017.

<img src="assets/encoder_decoder.png" alt="Encoder-Decoder Architecture" style="background-color:white;" height="500" />

### **Decoder-Only Architecture**

It is used for **sequence generation** tasks like language modeling.

This simpler architecture will be implemented in this notebook from scratch.

<img src="assets/decoder_only.png" alt="Decoder-Only Architecture" style="background-color:white;" height="500" />

In [1]:
import os

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

torch.manual_seed(42)
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

cuda


## **Scaled Dot-Product Attention**

Let $Q \in \mathbb{R}^{m \times d_k}$, $K \in \mathbb{R}^{n \times d_k}$ y $V \in \mathbb{R}^{n \times d_v}$ be the query, key and value matrices, respectively. The scaled dot-product attention is defined as:

$$
\text{Attention}(Q, K, V) = \text{softmax}\left(\frac{QK^T}{\sqrt{d_k}}\right)V \in \mathbb{R}^{m \times d_v}
$$

<img src="assets/self_attention.png" alt="Decoder-Only Architecture" style="background-color:white;" height="500" />

In [2]:
class ScaledDotProductAttention(nn.Module):

    def __init__(self, embeddingSize: int, headSize: int, blockSize: int, dropout: float) -> None:
        super().__init__()
        self.key = nn.Linear(embeddingSize, headSize, bias=False)
        self.query = nn.Linear(embeddingSize, headSize, bias=False)
        self.value = nn.Linear(embeddingSize, headSize, bias=False)
        self.register_buffer("tril", torch.tril(torch.ones(blockSize, blockSize)))
        self.dropout = nn.Dropout(dropout)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        B, T, C = x.shape
        k = self.key(x)
        q = self.query(x)
        v = self.value(x)
        weights = q @ k.transpose(-2, -1) / (C ** 0.5)
        weights = weights.masked_fill(self.tril[:T, :T] == 0, float("-inf"))
        weights = F.softmax(weights, dim=-1)
        weights = self.dropout(weights)
        output = weights @ v
        return output

## **Multi-Head Attention**

Let $h$ be the number of heads. The multi-head attention is defined as:

$$
\text{MultiHead}(Q, K, V) = \text{Concat}(\text{head}_1, \ldots, \text{head}_h)W^O \in \mathbb{R}^{m \times d_v}
$$

where $\text{head}_i = \text{Attention}(QW_i^Q, KW_i^K, VW_i^V)$ and $W_i^Q \in \mathbb{R}^{d_k \times d_k}$, $W_i^K \in \mathbb{R}^{d_k \times d_k}$, $W_i^V \in \mathbb{R}^{d_v \times d_v}$ and $W^O \in \mathbb{R}^{hd_v \times d_v}$ are the learnable parameters.

<img src="assets/multi_head_attention.png" alt="Decoder-Only Architecture" style="background-color:white;" height="500" />

In [3]:
class MultiHeadAttention(nn.Module):

    def __init__(self, nHeads: int, embeddingSize: int, headSize: int, blockSize: int, dropout: float) -> None:
        super().__init__()
        self.heads = nn.ModuleList([ScaledDotProductAttention(embeddingSize, headSize, blockSize, dropout) for _ in range(nHeads)])
        self.projection = nn.Linear(embeddingSize, embeddingSize)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        output = torch.cat([head(x) for head in self.heads], dim=-1)
        output = self.projection(output)
        return output

## **Position-wise Feed-Forward Networks**

The position-wise feed-forward networks are defined as:

$$
\text{FFN}(x) = \text{ReLU}(xW_1 + b_1)W_2 + b_2
$$

where $W_1 \in \mathbb{R}^{d_{ff} \times d_{model}}$, $b_1 \in \mathbb{R}^{d_{ff}}$, $W_2 \in \mathbb{R}^{d_{model} \times d_{ff}}$ and $b_2 \in \mathbb{R}^{d_{model}}$ are the learnable parameters.

In [4]:
class FeedForward(nn.Module):

    def __init__(self, embeddingSize: int, dropout: float) -> None:
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(embeddingSize, 4 * embeddingSize),
            nn.ReLU(),
            nn.Linear(4 * embeddingSize, embeddingSize),
            nn.Dropout(dropout)
        )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        return self.net(x)

## **Layer Normalization**

The layer normalization is defined as:

$$
\text{LayerNorm}(x) = \gamma \odot \frac{x - \mu}{\sigma} + \beta
$$

where $\gamma \in \mathbb{R}^{d_{model}}$ and $\beta \in \mathbb{R}^{d_{model}}$ are the learnable parameters, and $\mu$ and $\sigma$ are the mean and standard deviation of $x$, respectively.

In the original paper, the layer normalization is applied after the residual connections, whereas in more modern implementations, it is applied before in what is known as **pre-norm formulation**.

```python
# Original paper
x = LayerNorm(x + MultiHeadAttention(x))
x = LayerNorm(x + FeedForward(x))

# Pre-norm formulation
x = x + MultiHeadAttention(LayerNorm(x))
x = x + FeedForward(LayerNorm(x))
```

In [5]:
class Layer(nn.Module):

    def __init__(self, nHeads: int, embeddingSize: int, blockSize: int, dropout: float) -> None:
        super().__init__()
        headSize = embeddingSize // nHeads
        self.multiHeadAttention = MultiHeadAttention(nHeads, embeddingSize, headSize, blockSize, dropout)
        self.feedForward = FeedForward(embeddingSize, dropout)
        self.layerNorm1 = nn.LayerNorm(embeddingSize)
        self.layerNorm2 = nn.LayerNorm(embeddingSize)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = x + self.multiHeadAttention(self.layerNorm1(x))
        x = x + self.feedForward(self.layerNorm2(x))
        return x

## **Language Model**

In [6]:
class GPTLanguageModel(nn.Module):

    def __init__ (self, vocabularySize: int, nLayers: int, nHeads: int, embeddingSize: int, blockSize: int, dropout: float) -> None:
        super().__init__()
        self.tokenEmbeddingTable = nn.Embedding(vocabularySize, embeddingSize)
        self.positionEmbeddingTable = nn.Embedding(blockSize, embeddingSize)
        self.layers = nn.Sequential(*[Layer(nHeads, embeddingSize, blockSize, dropout) for _ in range(nLayers)])
        self.layerNorm = nn.LayerNorm(embeddingSize)
        self.linearModelHead = nn.Linear(embeddingSize, vocabularySize)
        self.blockSize = blockSize
        self.apply(self._init_weights)
    
    def _init_weights(self, module: nn.Module) -> None:
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)

    def forward(self, idx: torch.Tensor, targets: torch.Tensor = None) -> tuple[torch.Tensor, torch.Tensor]:
        B, T = idx.shape
        tokenEmbeddings = self.tokenEmbeddingTable(idx)
        positionEmbeddings = self.positionEmbeddingTable(torch.arange(T, device=device))
        x = tokenEmbeddings + positionEmbeddings
        x = self.layers(x)
        x = self.layerNorm(x)
        logits = self.linearModelHead(x)
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        return logits, loss

    def generate(self, idx: torch.Tensor, nTokens: int) -> torch.Tensor:
        self.eval()
        with torch.no_grad():
            for _ in range(nTokens):
                logits, loss = self(idx[:, -self.blockSize:])
                logits = logits[:, -1, :]
                probabilities = F.softmax(logits, dim=-1)
                nextIdx = torch.multinomial(probabilities, num_samples=1)
                idx = torch.cat([idx, nextIdx], dim=1)
        self.train()
        return idx

## **Text Generation**

## **Text Generation**

In [7]:
dataDir = "data"
file = os.path.join(dataDir, "shakespeare.txt")
with open(file, "r") as f:
    text = f.read()
print(f"Text length: {len(text)}")
print(text[:250])

Text length: 1115394
First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.



In [8]:
vocabulary = sorted(set(text))
vocabularySize = len(vocabulary)
print(f"Vocabulary size: {vocabularySize}")
print("".join(vocabulary))

Vocabulary size: 65

 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz


In [9]:
stringToToken = {ch: i for i, ch in enumerate(vocabulary)}
tokenToString = {i: ch for i, ch in enumerate(vocabulary)}
encode = lambda string: [stringToToken[char] for char in string]
decode = lambda tokens: "".join([tokenToString[token] for token in tokens])
print(encode("To be or not to be"))
print(decode(encode("To be or not to be")))

[32, 53, 1, 40, 43, 1, 53, 56, 1, 52, 53, 58, 1, 58, 53, 1, 40, 43]
To be or not to be


In [12]:
blockSize = 256
embeddingSize = 384
nHeads = 6
nLayers = 6
dropout = 0.2

model = GPTLanguageModel(vocabularySize, nLayers, nHeads, embeddingSize, blockSize, dropout).to(device)
print(f"{sum(p.numel() for p in model.parameters())/1e6:.2f}M parameters")

10.79M parameters


In [13]:
modelsDir = "models"
model.load_state_dict(torch.load(os.path.join(modelsDir, "shakespeare.pt"), weights_only=False))

<All keys matched successfully>

In [14]:
context = "To be or not to be"
context = torch.tensor(encode(context), dtype=torch.long).unsqueeze(0).to(device)
generatedText = decode(model.generate(context, nTokens=10000)[0].tolist())
print(generatedText)
outputDir = "outputs"
with open(os.path.join(outputDir, "shakespeare.txt"), "w") as f:
    f.write(generatedText)

To be or not to bed,
His affairs plucks and men it and heart:
Therefore, my lords, King Edward we speak,
Speak, must I near, be not die:
The contriuments that I saw thy request;
For that, not shall hear their wull hich shows them,
And she may show, methinks me here in her;
And with their senest upon my heart came
From God tears me with disholopeful held.
As so the lungly correts and wilt the resemble
And would not what may do such a lost.
There's Tybalt Angelo, but was traitor!
I must ake the lands, and thinking these rogues,
Ny Ebeseemino; mark, I cannot have done:
I, this say; long on tarry, for thou art,
Thou hast she pure beholder'd; kill me was butt,
So set let me up, and I burn for this.

JULIET:
Marry, look you, my lord. I'll had you tell you of Weep
What news?' what flesh is true than o'er company?

ROMEO:
Dear belike? friar, ere think you not: not show you
Say have made. Good mars, what a tympassion?
Pack, go the man for a lamb. For a lame, and fife;
not to look me o' the woe,